In [1]:
import ase
import torch
import torchani

In [2]:
from ase.io import read,write

In [4]:
molecule = read('phenol-OPTED.com', format='gaussian')

In [5]:
model = torchani.models.ANI1x().double()

In [6]:
molecule.set_calculator(model.ase())

## Compute Energy Using ANI

In [7]:
species = model.species_to_tensor(molecule.get_chemical_symbols()).unsqueeze(0)
coordinates = torch.from_numpy(molecule.get_positions()).unsqueeze(0).requires_grad_(True)


energy = model((species, coordinates)).energies
print(energy.item())

-307.38023581599185


## Compute Energy Using ASE & ANI Potential

In [8]:
En = molecule.get_potential_energy()
print(En,'eV')

-8364.242253049988 eV


In [9]:
from ase.units import Hartree
print('Energy as hartee:',En/Hartree)

Energy as hartee: -307.38023581599185


## Calculate Frequencies Using ANI

In [10]:
species = model.species_to_tensor(molecule.get_chemical_symbols()).unsqueeze(0)
coordinates = torch.from_numpy(molecule.get_positions()).unsqueeze(0).requires_grad_(True)


element_masses = torch.tensor([
    1.008,  # H
    12.011,  # C
    14.007,  # N
    15.999,  # O
], dtype=torch.double)
masses = element_masses[species]

In [11]:
energies = model((species, coordinates)).energies
hessian = torchani.utils.hessian(coordinates, energies=energies)

In [12]:
freq, modes = torchani.utils.vibrational_analysis(masses, hessian)
# freq = freq[-3:]
# modes = modes[-3:]

print('Frequencies (cm^-1):', freq)
# print('Modes:', modes)

Frequencies (cm^-1): tensor([      nan,       nan,       nan,   22.1375,   27.0655,   33.7213,
         240.9058,  323.9623,  398.5963,  433.3237,  505.9335,  549.0664,
         641.2338,  654.6740,  792.5240,  821.3046,  857.1333,  894.8374,
         943.2652,  972.4365, 1057.9421, 1084.0008, 1133.6824, 1194.4545,
        1226.6036, 1249.0623, 1324.2374, 1350.5656, 1408.5865, 1555.6388,
        1569.4365, 1713.7795, 1751.0878, 3242.1944, 3261.7901, 3271.2612,
        3282.7133, 3299.0188, 3785.8384], dtype=torch.float64)


## Calculate Frequencies Using ASE & ANI Potential

In [13]:
from ase.vibrations import Vibrations

In [14]:
vib = Vibrations(molecule)

In [15]:
vib.run()

Writing vib.eq.pckl
Writing vib.0x-.pckl
Writing vib.0x+.pckl
Writing vib.0y-.pckl
Writing vib.0y+.pckl
Writing vib.0z-.pckl
Writing vib.0z+.pckl
Writing vib.1x-.pckl
Writing vib.1x+.pckl
Writing vib.1y-.pckl
Writing vib.1y+.pckl
Writing vib.1z-.pckl
Writing vib.1z+.pckl
Writing vib.2x-.pckl
Writing vib.2x+.pckl
Writing vib.2y-.pckl
Writing vib.2y+.pckl
Writing vib.2z-.pckl
Writing vib.2z+.pckl
Writing vib.3x-.pckl
Writing vib.3x+.pckl
Writing vib.3y-.pckl
Writing vib.3y+.pckl
Writing vib.3z-.pckl
Writing vib.3z+.pckl
Writing vib.4x-.pckl
Writing vib.4x+.pckl
Writing vib.4y-.pckl
Writing vib.4y+.pckl
Writing vib.4z-.pckl
Writing vib.4z+.pckl
Writing vib.5x-.pckl
Writing vib.5x+.pckl
Writing vib.5y-.pckl
Writing vib.5y+.pckl
Writing vib.5z-.pckl
Writing vib.5z+.pckl
Writing vib.6x-.pckl
Writing vib.6x+.pckl
Writing vib.6y-.pckl
Writing vib.6y+.pckl
Writing vib.6z-.pckl
Writing vib.6z+.pckl
Writing vib.7x-.pckl
Writing vib.7x+.pckl
Writing vib.7y-.pckl
Writing vib.7y+.pckl
Writing vib.7z

In [16]:
vib.combine()

79

In [17]:
vib.summary()

---------------------
  #    meV     cm^-1
---------------------
  0    0.5i      4.0i
  1    0.2i      2.0i
  2    0.0i      0.0i
  3    2.8      22.5 
  4    3.2      26.0 
  5    4.2      34.2 
  6   29.9     241.0 
  7   40.3     325.1 
  8   49.4     398.8 
  9   53.9     434.4 
 10   62.7     506.0 
 11   68.2     550.1 
 12   79.5     641.5 
 13   81.2     654.8 
 14   98.3     792.8 
 15  101.9     821.5 
 16  107.2     864.7 
 17  111.0     895.0 
 18  117.0     943.5 
 19  120.6     972.7 
 20  131.6    1061.8 
 21  134.8    1087.1 
 22  141.0    1137.4 
 23  148.2    1195.1 
 24  152.2    1227.5 
 25  154.9    1249.7 
 26  165.0    1330.5 
 27  167.5    1350.9 
 28  174.7    1409.3 
 29  193.3    1559.2 
 30  195.0    1572.8 
 31  213.2    1719.5 
 32  217.3    1752.9 
 33  404.0    3258.6 
 34  404.3    3260.6 
 35  406.0    3274.3 
 36  407.5    3286.9 
 37  409.4    3302.2 
 38  469.5    3786.8 
---------------------
Zero-point energy: 2.910 eV


In [18]:
vib.write_jmol()

/home/saeed/.local/lib/python3.7/site-packages/ase/vibrations/vibrations.py:469: ComplexWarning: Casting complex values to real discards the imaginary part
  fd.write('Mode #%d, f = %.1f%s cm^-1' % (n, f[n], c))
